In [2]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load configuration from config.json
with open('../models/config.json', 'r') as f:
    config = json.load(f)

postprocess = config['postprocessing']

# Pfad zum Verzeichnis für die Dateien
INPUT_PATH_HD = postprocess['input_path_hd']

dates = ['20230316', '20230328', '20230403', '20230409', '20230410', '20230501', '20230515', '20230619', '20230622', '20230626', '20230709', '20230710']

# Output path for plots (create if doesn't exist)
OUTPUT_PATH = 'output_plots'
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

def process_date(date):
    start_date = date
    end_date = date

    # Pfade zu den Dateien
    actual_hd_file = f'{INPUT_PATH_HD}/actual_heat_demand_{start_date}_to_{end_date}_day.json'
    forecast_hd_file = f'{INPUT_PATH_HD}/heat_demand_{start_date}_to_{end_date}_day.json'
    scenarios_hd_file = f'{INPUT_PATH_HD}/reduced_heat_demand_scenarios_{start_date}_to_{end_date}_day.json'
    weighted_hd_file = f'{INPUT_PATH_HD}/weighted_heat_demand/weighted_heat_demand_{start_date}.json'

    # Einlesen der prognostizierten Wärmeanforderung
    with open(forecast_hd_file, 'r') as f:
        forecast_data = json.load(f)
    forecast_hd_values = forecast_data['heat_demand']
    forecast_hd_series = pd.Series({int(k): float(v) for k, v in forecast_hd_values.items()})
    forecast_hd_series = forecast_hd_series.sort_index()

    # Berechne die prognostizierte Gesamtwärmeanforderung
    total_forecast_hd = forecast_hd_series.sum()

    # Einlesen der Wärmeanforderungen der Szenarien
    with open(scenarios_hd_file, 'r') as f:
        scenarios_data = json.load(f)

    # Optional: Einlesen der tatsächlichen Wärmeanforderung (falls vorhanden)
    actual_hd_available = False
    try:
        with open(actual_hd_file, 'r') as f:
            actual_data = json.load(f)
        actual_hd_values = actual_data['heat_demand']
        actual_hd_series = pd.Series({int(k): float(v) for k, v in actual_hd_values.items()})
        actual_hd_series = actual_hd_series.sort_index()
        total_actual_hd = actual_hd_series.sum()
        actual_hd_available = True
    except FileNotFoundError:
        print(f"Keine tatsächliche Wärmeanforderung für {date} vorhanden.")

    # Optional: Einlesen der gewichteten Wärmeanforderung (weighted heat demand)
    weighted_hd_available = False
    try:
        with open(weighted_hd_file, 'r') as f:
            weighted_data = json.load(f)
        weighted_hd_values = weighted_data['heat_demand']
        weighted_hd_series = pd.Series({int(k): float(v) for k, v in weighted_hd_values.items()})
        weighted_hd_series = weighted_hd_series.sort_index()
        total_weighted_hd = weighted_hd_series.sum()
        weighted_hd_available = True
    except FileNotFoundError:
        print(f"Keine gewichtete Wärmeanforderung für {date} vorhanden.")
        total_weighted_hd = None

    # Berechnung der Differenzen und der gewichteten Gesamtwärmedifferenzen
    weighted_differences = []
    scenario_names = []
    total_differences = []
    probabilities = []

    for scenario_name, scenario_info in scenarios_data.items():
        probability = scenario_info['Probability']
        probabilities.append(probability)
        scenario_names.append(scenario_name)
        # Entferne 'Probability' aus dem Dictionary, um die stündlichen Wärmeanforderungen zu erhalten
        scenario_hd_values = {int(k): float(v) for k, v in scenario_info.items() if k != 'Probability'}
        scenario_hd_series = pd.Series(scenario_hd_values).sort_index()
        # Berechne die stündlichen Differenzen zur Prognose
        differences_series = forecast_hd_series - scenario_hd_series
        # Summiere die Differenzen über alle Stunden
        print(f"Differenzen für Szenario {scenario_name}: {differences_series}")
        total_difference = differences_series.sum()
        total_differences.append(total_difference)
        # Berechne die gewichtete Differenz
        weighted_difference = probability * total_difference
        weighted_differences.append(weighted_difference)
    
    print(f"Weighted differences: {weighted_differences}")
    # Berechnung der erwarteten Gesamtwärmedifferenz
    expected_total_difference = sum(weighted_differences)
    # Berechnung der erwarteten Gesamtwärmeanforderung des stochastischen Modells
    expected_total_hd = total_forecast_hd - expected_total_difference

    # Ausgabe der Ergebnisse
    print(f"\nDatum: {date}")
    print(f"Prognostizierte Gesamtwärmeanforderung: {total_forecast_hd:.2f} kWh")
    print(f"Erwartete Gesamtwärmedifferenz: {expected_total_difference:.2f} kWh")
    print(f"Erwartete Gesamtwärmeanforderung (Stochastisches Modell): {expected_total_hd:.2f} kWh")

    if weighted_hd_available:
        print(f"Gewichtete Gesamtwärmeanforderung: {total_weighted_hd:.2f} kWh")

    if actual_hd_available:
        print(f"Tatsächliche Gesamtwärmeanforderung: {total_actual_hd:.2f} kWh")

    # Überprüfung der Summe der Wahrscheinlichkeiten
    total_probability = sum(probabilities)
    print(f"Summe der Wahrscheinlichkeiten: {total_probability:.4f}")

    # Tabelle mit den Szenarien und ihren Gesamtwärmedifferenzen
    scenarios_df = pd.DataFrame({
        'Szenario': scenario_names,
        'Wahrscheinlichkeit': probabilities,
        'Gesamtwärmedifferenz [kWh]': total_differences,
        'Gewichtete Differenz [kWh]': weighted_differences
    })
    print("\nSzenarien und ihre Gesamtwärmedifferenzen:")
    print(scenarios_df)

    # Modelle und ihre Gesamtwärmeanforderungen
    models = ['Prognose', 'Stochastisch']
    total_hds = [total_forecast_hd, expected_total_hd]

    if weighted_hd_available:
        models.append('Gewichtet')
        total_hds.append(total_weighted_hd)

    if actual_hd_available:
        models.append('Tatsächlich')
        total_hds.append(total_actual_hd)

    # Farben für die Balken festlegen
    colors = ['blue', 'orange']
    if weighted_hd_available:
        colors.append('purple')
    if actual_hd_available:
        colors.append('green')

    # Neuer Plot erstellen
    fig, ax = plt.subplots(figsize=(8, 6))
    bars = ax.bar(models, total_hds, color=colors)
    ax.set_ylabel('Gesamtwärmeanforderung [kWh]', fontsize=14)
    ax.tick_params(axis='both', labelsize=12)

    # Werte über den Balken anzeigen
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.1f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=12)

    plt.tight_layout()
    # Plot speichern
    plot_filename = f'{OUTPUT_PATH}/waermeanforderungen_{date}.png'
    plt.savefig(plot_filename)
    plt.close(fig)  # Close the figure to free memory

    # Rückgabe der Ergebnisse
    return {
        'date': date,
        'total_forecast_hd': total_forecast_hd,
        'expected_total_hd': expected_total_hd,
        'total_weighted_hd': total_weighted_hd,
        'total_actual_hd': total_actual_hd if actual_hd_available else None,
        'scenarios_df': scenarios_df
    }

# Hauptprogramm
results = []

for date in dates:
    result = process_date(date)
    results.append(result)

# Zusammenfassung der Ergebnisse
summary_data = {
    'date': [],
    'total_forecast_hd': [],
    'expected_total_hd': [],
    'total_weighted_hd': [],
    'total_actual_hd': []
}

for result in results:
    summary_data['date'].append(result['date'])
    summary_data['total_forecast_hd'].append(result['total_forecast_hd'])
    summary_data['expected_total_hd'].append(result['expected_total_hd'])
    summary_data['total_weighted_hd'].append(result['total_weighted_hd'])
    summary_data['total_actual_hd'].append(result['total_actual_hd'])

summary_df = pd.DataFrame(summary_data)
print("\nZusammenfassung der Ergebnisse:")
print(summary_df)


Differenzen für Szenario Scenario1: 1     12.731658
2     30.821550
3     -4.289053
4     11.219356
5     20.959325
6     29.400375
7    -12.923747
8     35.443727
9      4.254730
10   -19.278714
11    16.937170
12   -11.592747
13    14.417150
14    11.036740
15    39.630386
16    12.303575
17     7.271780
18     4.348450
19    20.399162
20    26.234308
21    38.482689
22    36.363060
23    21.101102
24    -1.574924
dtype: float64
Differenzen für Szenario Scenario2: 1     55.620949
2      8.717702
3     -4.289053
4     11.219356
5     -4.661963
6    -23.099685
7     11.665525
8      8.965185
9      4.254730
10     9.849401
11   -10.468257
12    11.808680
13    14.417150
14    34.226773
15    39.630386
16    12.303575
17     7.271780
18   -23.481731
19    20.399162
20   -14.722570
21    20.181567
22    19.313459
23    21.101102
24    38.852625
dtype: float64
Differenzen für Szenario Scenario3: 1     12.731658
2    -13.386146
3    -38.628091
4     11.219356
5     20.959325
6     29.40037